In [3]:
import pandas as pd
from gensim.utils import simple_preprocess
import sklearn
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,Embedding,Conv1D,Flatten
from sklearn.metrics import confusion_matrix,accuracy_score

Using TensorFlow backend.


In [4]:
df = pd.read_csv("/home/shiva/Downloads/datasets/WK_1.csv", compression='gzip')

In [5]:
df.head()

,Unnamed: 0,cleaned_data,Administrative Law,Attorneys Fees,Banking Law,Bankruptcy Law,Civil Procedure,Civil Rights,Class Actions,Commercial Law,...,Immigration Law,Injury & Tort Law,Insurance Law,Intellectual Property,Labor & Employment Law,Property Law & Real Estate,Remedies,Sanctions,Securities Law,Sentencing
0,0,immaterial exception pendency motion suit dist...,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,1,consideration motion opinion case slip opinion...,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,2,consideration motion opinion case slip opinion...,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,3,terms time dismissal labor organization IAWCs ...,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,4,Plaintiffs limited ambulatory surgical center ...,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df['word_len']=df.cleaned_data.apply(lambda x:len(simple_preprocess(x)))

In [7]:
# plt.figure(figsize=(100,25))
# sns.barplot(x=np.arange(len(df)),y=word_count)

In [8]:
data=df.loc[df.word_len>=50]
data.head(2)

,Unnamed: 0,cleaned_data,Administrative Law,Attorneys Fees,Banking Law,Bankruptcy Law,Civil Procedure,Civil Rights,Class Actions,Commercial Law,...,Injury & Tort Law,Insurance Law,Intellectual Property,Labor & Employment Law,Property Law & Real Estate,Remedies,Sanctions,Securities Law,Sentencing,word_len
0,0,immaterial exception pendency motion suit dist...,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,614
3,3,terms time dismissal labor organization IAWCs ...,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,629


In [9]:
data.cleaned_data=data.cleaned_data.apply(lambda x:x.lower())

/home/shiva/python_projects_directory/python_projects_env/lib/python3.5/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [10]:
labels=[[j for j in data.iloc[i,2:-1].values] for i in range(len(data))]
labels=np.array(labels)
len(labels)

5755

In [11]:
vect=Tokenizer()
vect.fit_on_texts(data['cleaned_data'])
vocab_size = len(vect.word_index)+1
print(vocab_size)
encoded_docs_train = vect.texts_to_sequences(data['cleaned_data'])
max_length = vocab_size
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=200, padding='post')
# padded_docs_train1=padded_docs_train.reshape((5755,1,200))

31121


In [12]:
# n_classes=len(labels[0])
# model = Sequential()
# model.add(Conv1D(128,1,input_shape=(1,200)))  
# # Adding a dropout layer
# model.add(Dropout(0.2))
# model.add(Conv1D(64,1))
# model.add(Dropout(0.2))
# model.add(Flatten())
# # Adding a dense output layer with sigmoid activation
# model.add(Dense(n_classes, activation='softmax'))
# model.summary()

In [13]:
n_classes=len(labels[0])
model = Sequential()
# Configuring the parameters
model.add(Embedding(vocab_size, output_dim=128, input_length=200))
model.add(LSTM(128, return_sequences=True))  
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 128)          3983488   
_________________________________________________________________
lstm_1 (LSTM)                (None, 200, 128)          131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 31)                2015      
Total params: 4,166,495
Trainable params: 4,166,495
Non-trainable params: 0
____________________________________________

In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
history = model.fit(padded_docs_train, labels,
                    class_weight='balanced',
                    epochs=1,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=[])

/home/shiva/python_projects_directory/python_projects_env/lib/python3.5/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 5179 samples, validate on 576 samples
Epoch 1/1
1184/5179 [=====>........................] - ETA: 43s - loss: 0.2779 - accuracy: 0.9228

KeyboardInterrupt: 

In [116]:
test = vect.texts_to_sequences([input().lower()])
test = pad_sequences(test, maxlen=200, padding='post')
print(test.shape)
pred=model.predict(test)

TV operators contracts gross revenues service parties cable modem franchised cable phrase quotations modem analog signals service issue digital common usage phrase cable broadband Internet service cable television signals cable modem service telecommunication Internet cable operators payments cable modem services franchise fees cable operator respect cable system percent cable operators gross revenues period operation cable system cable services cable modem service programming reasoning cable operators role Internet service gross revenues period operation cable system cable services emphasis fee revenues TV service statutory cap cities fees gross revenue services income allowable revenue base FCCs understanding difference services operators parties cable services programming services cable modem service consequences definition products statutes domain governments unlimited fees tax statutes revenue uses cable cable operators statute city franchise fee respect cable system total payment

In [27]:
df.cleaned_data[3456]

'TV operators contracts gross revenues service parties cable modem franchised cable phrase quotations modem analog signals service issue digital common usage phrase cable broadband Internet service cable television signals cable modem service telecommunication Internet cable operators payments cable modem services franchise fees cable operator respect cable system percent cable operators gross revenues period operation cable system cable services cable modem service programming reasoning cable operators role Internet service gross revenues period operation cable system cable services emphasis fee revenues TV service statutory cap cities fees gross revenue services income allowable revenue base FCCs understanding difference services operators parties cable services programming services cable modem service consequences definition products statutes domain governments unlimited fees tax statutes revenue uses cable cable operators statute city franchise fee respect cable system total paymen

In [117]:
pred.ravel()

array([0.02470733, 0.03135265, 0.02907716, 0.0418697 , 0.06844533,
       0.03602467, 0.04274934, 0.03745964, 0.02363819, 0.02315982,
       0.10841313, 0.02355727, 0.01005258, 0.02828099, 0.03660313,
       0.03005597, 0.01226931, 0.02096683, 0.0251615 , 0.01315554,
       0.037026  , 0.01540057, 0.03637374, 0.04926513, 0.02020056,
       0.06953914, 0.02361884, 0.02012124, 0.02330609, 0.02850833,
       0.00964023], dtype=float32)

In [118]:
def sigmoid(scores):
    scores = np.negative(scores)
    exp = np.exp(scores)
    scores = 1 / (1 + exp)
    scores=[1 if i>0.51 else 0 for i in scores.ravel()]
    return scores
result=sigmoid(pred)

In [119]:
for i,j in zip(result,labels[3456]):
    print(i,'  ',j)

0    0
0    0
0    0
1    0
1    0
0    0
1    0
0    0
0    1
0    0
1    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
1    0
0    0
1    0
0    0
0    0
0    0
0    0
0    0


In [52]:
categories={'cat'+str(i):cat for i,cat in enumerate(list(data.head(0))[2:-1])}

# ml algos for multilabel

In [36]:
# using binary relevance
from sklearn.pipeline import make_pipeline 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.ensemble import RandomForestClassifier
xtrain,xtest,ytrain,ytest=train_test_split(data.cleaned_data,labels,test_size=0.2,random_state=50)
classifier=BinaryRelevance(RandomForestClassifier(min_samples_leaf=1, min_samples_split=6, n_estimators=120, 
                             criterion='gini', bootstrap='False', n_jobs= -1))
pipe = make_pipeline(TfidfVectorizer(ngram_range=(1,2),norm='l2',max_features=500),classifier)
pipe.fit(xtrain,ytrain)
from imblearn.metrics import  classification_report_imbalanced 
y_pred = pipe.predict(xtest)
# probablity=pipe.predict_proba(X_test_JP)
# Model Accuracy
print("Random forest Accuracy:", accuracy_score(ytest, y_pred))


Random forest Accuracy: 0.3840139009556907


In [37]:
y_pred=y_pred.toarray()
from sklearn.metrics import precision_score,recall_score,f1_score
label_bin={'cat'+str(j):[labels[cat][j] for cat in range(len(ytest))] for j in range(len(ytest[0]))}
pred_bin={'cat'+str(j):[y_pred[cat][j] for cat in range(len(y_pred))] for j in range(len(y_pred[0]))}

In [43]:
from imblearn.metrics import  classification_report_imbalanced 
report=classification_report_imbalanced(label_bin['cat7'],pred_bin['cat7'])
print(report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.96      1.00      0.00      0.98      0.00      0.00      1108
          1       0.00      0.00      1.00      0.00      0.00      0.00        43

avg / total       0.93      0.96      0.04      0.94      0.00      0.00      1151



In [65]:
df.cleaned_data[5000]

'employer year period order significant debt excessive purchases jewelry debt husband guilty count wire fraud violation sentencing district court motion downward departure applicable sentencing range offense mental capacity district court discretion departure sentence case resentencing chronic depression adult life unnecessary shopping pain depression years psychiatric therapy behaviorﾗwhich doctors compulsive shoppingﾗand depression Roachs shopping binges severe strain marriage behavior binges husband credit card statements friends homes entries checkbook husband credit cards new ones annual income considerable condominium fashionable neighborhoods thousands dollars credit card debt purchases jewelry upscale stores occasion store credit card terrified husband extent debts fraud experienced manager associate partner annual salary expense report reimbursement conference registration fees personal charge card unable conference expense request fees money money opportunity debt debt husban

In [66]:
pred=pipe.predict([input()])
pred=pred.toarray()
for i,j in zip(labels[5000],pred):
    print(i,j)

employer year period order significant debt excessive purchases jewelry debt husband guilty count wire fraud violation sentencing district court motion downward departure applicable sentencing range offense mental capacity district court discretion departure sentence case resentencing chronic depression adult life unnecessary shopping pain depression years psychiatric therapy behaviorﾗwhich doctors compulsive shoppingﾗand depression Roachs shopping binges severe strain marriage behavior binges husband credit card statements friends homes entries checkbook husband credit cards new ones annual income considerable condominium fashionable neighborhoods thousands dollars credit card debt purchases jewelry upscale stores occasion store credit card terrified husband extent debts fraud experienced manager associate partner annual salary expense report reimbursement conference registration fees personal charge card unable conference expense request fees money money opportunity debt debt husband

0 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


In [67]:
print(labels[5000])
print(pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]


In [33]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain

classifier = ClassifierChain(RandomForestClassifier(min_samples_leaf=1, min_samples_split=6, n_estimators=120, 
                             criterion='gini', bootstrap='False', n_jobs= -1))
pipe = make_pipeline(TfidfVectorizer(ngram_range=(1,2),norm='l2',max_features=500),classifier)
pipe.fit(data.cleaned_data,labels)
from imblearn.metrics import  classification_report_imbalanced 
y_pred = pipe.predict(data.cleaned_data)
# probablity=pipe.predict_proba(X_test_JP)
# Model Accuracy
print("Random forest Accuracy:", accuracy_score(labels, y_pred))


NameError: name 'RandomForestClassifier' is not defined

In [88]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(data.cleaned_data,labels,test_size=0.2,random_state=50)
classifier = LabelPowerset(RandomForestClassifier(min_samples_leaf=1, min_samples_split=6, n_estimators=120, 
                             criterion='gini', bootstrap='False', n_jobs= -1))
pipe = make_pipeline(TfidfVectorizer(ngram_range=(1,2),norm='l2',max_features=500),classifier)
pipe.fit(xtrain,ytrain)
from imblearn.metrics import  classification_report_imbalanced 
y_pred = pipe.predict(xtest)
# probablity=pipe.predict_proba(X_test_JP)
# Model Accuracy
print("Random forest Accuracy:", accuracy_score(ytest, y_pred))

Random forest Accuracy: 0.45091225021720244


In [78]:
# import pickle
# pickle.dump(pipe,open('legal_model.pkl','wb'))
# pipe=pickle.load(open('legal_model.pkl','rb'))

In [80]:
y_pred=y_pred.toarray()
from sklearn.metrics import precision_score,recall_score,f1_score
label_bin={'cat'+str(j):[labels[cat][j] for cat in range(len(ytest))] for j in range(len(ytest[0]))}
pred_bin={'cat'+str(j):[y_pred[cat][j] for cat in range(len(y_pred))] for j in range(len(y_pred[0]))}

In [85]:
# from imblearn.metrics import  classification_report_imbalanced 
# classification_report=[classification_report_imbalanced(label_bin['cat'+str(j)],pred_bin['cat'+str(j)]) for j in range(len(labels[0]))]
# for i,p in enumerate(classification_report):
#     print('-----------------------********************{}******************---------------------------'.format(categories['cat'+str(i)]))
#     print(p)
    

In [87]:
classification_report_imbalanced(cat1,pred1)

'                   pre       rec       spe        f1       geo       iba       sup\n\n          0       1.00      1.00      0.95      1.00      0.98      0.96      1064\n          1       0.98      0.95      1.00      0.97      0.98      0.95        87\n\navg / total       0.99      0.99      0.96      0.99      0.98      0.96      1151\n'

In [32]:
from skmultilearn.adapt import MLkNN

classifier = MLkNN(k=20)
pipe = make_pipeline(TfidfVectorizer(ngram_range=(1,2),norm='l2',max_features=500),classifier)
pipe.fit(data.cleaned_data,labels)
from imblearn.metrics import  classification_report_imbalanced 
y_pred = pipe.predict(data.cleaned_data)
# probablity=pipe.predict_proba(X_test_JP)
# Model Accuracy
print(" Accuracy:", accuracy_score(labels, y_pred))

 Accuracy: 0.3364031277150304
